In [1]:
# Importing libraries

from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
%matplotlib inline
import shim as ced
import matplotlib.image as mpimg

In [2]:
# Loading the data from the drive and saving it to Categories
DATADIR ="C:/study_material/capp/retina/1 vs 2,3,4"
CATEGORIES = ['1','2,3,4']

In [5]:
from shim_1 import *
IMG_SIZE=224
img_r=[]
y_train=[]
for category in CATEGORIES:
    path = os.path.join(DATADIR,category)
    class_num = CATEGORIES.index(category)
    imgs = load_data(path)
    for i in imgs:
        a= cv2.resize(i,(IMG_SIZE,IMG_SIZE))
        img_r.append(a)
        y_train.append(class_num)

detector = ced.cannyEdgeDetector(img_r, sigma=1.4, kernel_size=5, lowthreshold=0.071, highthreshold=0.12, weak_pixel=100)
imgs_final = detector.detect()

In [6]:
X_train=[]
a=np.array(imgs_final)
for image in a:
    image = tf.expand_dims(image, -1)
    x1 = image.shape[0]
    x2 = image.shape[1]
    image = tf.reshape(tf.broadcast_to(image, (x1, x2, 3)),  (x1, x2, 3))
    X_train.append(image)

In [9]:
# normalising the data
X_train = np.array(X_train)/255
y_train=np.array(y_train)

In [11]:
print(X_train.shape)
y_train.shape

(2386, 224, 224, 3)


(2386,)

In [12]:
#Laoding Val_Test data
DATADIR ="C:/study_material/capp/retina/Val_Test(1 vs 234)"
CATEGORIES = ['1','2,3,4']

from shim_1 import *
IMG_SIZE=224
img_r=[]
y_test=[]
for category in CATEGORIES:
    path = os.path.join(DATADIR,category)
    class_num = CATEGORIES.index(category)
    imgs = load_data(path)
    for i in imgs:
        a= cv2.resize(i,(IMG_SIZE,IMG_SIZE))
        img_r.append(a)
        y_test.append(class_num)

detector = ced.cannyEdgeDetector(img_r, sigma=1.4, kernel_size=5, lowthreshold=0.071, highthreshold=0.12, weak_pixel=100)
imgs_final = detector.detect()

In [14]:
# empty lists to append images and labels
X_test=[]
a=np.array(imgs_final)
for image in a:
    image = tf.expand_dims(image, -1)
    x1 = image.shape[0]
    x2 = image.shape[1]
    image = tf.reshape(tf.broadcast_to(image, (x1, x2, 3)),  (x1, x2, 3))
    X_test.append(image)



# converting the train and test data into arrays
X_test = np.array(X_test)/255

In [67]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras import Model
from tensorflow.keras.optimizers import RMSprop, Adam, SGD

In [69]:
pre_trained_model = VGG16(input_shape = (224, 224, 3), # Shape of our images
                                include_top = False, # Leave out the last fully connected layer
                                weights = None)

In [72]:
last_output = pre_trained_model.output
x=layers.GlobalAveragePooling2D()(last_output)
# Flatten the output layer to 1 dimension
#x = layers.Flatten()(x)
# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = layers.Dense(512, activation='tanh')(x)

# Add a dropout rate of 0.2
#x = layers.Dropout(0.2)(x)                  
# Add a final sigmoid layer for classification
x = layers.Dense  (1, activation='sigmoid')(x)           

cnn = Model( pre_trained_model.input, x) 

cnn.compile(optimizer = RMSprop(learning_rate=0.0001), 
              loss = 'binary_crossentropy', 
              metrics = ['acc'])

In [73]:
cnn.fit(np.array(X_train), np.array(y_train), epochs=20)

Epoch 1/20
75/75 [==============================] - 669s 9s/step - loss: 0.6429 - acc: 0.6429
Epoch 2/20
75/75 [==============================] - 652s 9s/step - loss: 0.6036 - acc: 0.6995
Epoch 3/20
75/75 [==============================] - 653s 9s/step - loss: 0.5725 - acc: 0.7179
Epoch 4/20
75/75 [==============================] - 669s 9s/step - loss: 0.5577 - acc: 0.7293
Epoch 5/20
75/75 [==============================] - 678s 9s/step - loss: 0.5204 - acc: 0.7552
Epoch 6/20
75/75 [==============================] - 567s 8s/step - loss: 0.5054 - acc: 0.7586
Epoch 7/20
75/75 [==============================] - 568s 8s/step - loss: 0.4760 - acc: 0.7854
Epoch 8/20
75/75 [==============================] - 582s 8s/step - loss: 0.4638 - acc: 0.7896
Epoch 9/20
75/75 [==============================] - 583s 8s/step - loss: 0.4420 - acc: 0.8051
Epoch 10/20
75/75 [==============================] - 648s 9s/step - loss: 0.4302 - acc: 0.8198
Epoch 11/20
75/75 [==============================] - 680s 9

In [74]:
cnn.save('sg_1.h5')

In [75]:
cnn.evaluate(np.array(X_test),np.array(y_test))

6/6 [==============================] - 15s 2s/step - loss: 1.1430 - acc: 0.6857


[1.1430062055587769, 0.6857143044471741]

In [76]:
y_pred = cnn.predict(X_test)
y_pred[:20]

array([[0.47761655],
       [0.0809128 ],
       [0.11020881],
       [0.00261593],
       [0.02425665],
       [0.59811187],
       [0.50462455],
       [0.11930764],
       [0.00644857],
       [0.00819987],
       [0.7583698 ],
       [0.00135016],
       [0.20106345],
       [0.00491241],
       [0.01110217],
       [0.48705393],
       [0.01762554],
       [0.44812894],
       [0.79558814],
       [0.00353453]], dtype=float32)

In [83]:
from sklearn.metrics import confusion_matrix , classification_report
import numpy as np
y_pred = cnn.predict(X_test)
y_pred_class=[]
for i in y_pred:
    if i>0.5:
        y_pred_class.append(1)
    else:
        y_pred_class.append(0) 

print("Classification Report: \n", classification_report(y_test, y_pred_class))

Classification Report: 
               precision    recall  f1-score   support

           0       0.30      0.78      0.43        27
           1       0.94      0.67      0.78       148

    accuracy                           0.69       175
   macro avg       0.62      0.72      0.61       175
weighted avg       0.84      0.69      0.73       175



In [84]:
print("Classification Report: \n", confusion_matrix(y_test, y_pred_class))

Classification Report: 
 [[21  6]
 [49 99]]
